In [1]:
import os
import glob
import math
import time
import pandas as pd
import numpy as np
import scipy as sc
from sklearn.model_selection import KFold
import warnings
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import random
import seaborn as sns; sns.set_theme()
import torch.nn.functional as F
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from matplotlib.pyplot import figure
from IPython import display
from pandas.plotting import scatter_matrix
from sklearn.metrics import r2_score
from sklearn import svm
from numpy import std
from numpy import mean
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from matplotlib import cm
from sklearn.metrics import confusion_matrix

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
warnings.filterwarnings('ignore')
pd.set_option('max_columns', 300)

In [4]:
train_test_seurat = pd.read_csv('./integrate.csv')

In [5]:
train_test_seurat = train_test_seurat.T

In [6]:
train_test_seurat_std = train_test_seurat.std()
column_names = list(train_test_seurat.columns)
columns_remove = []
for i in range(train_test_seurat.shape[1]):
    if train_test_seurat_std[i] == 0:
        columns_remove.append(column_names[i])

In [7]:
train_test_seurat = train_test_seurat.drop(columns_remove, axis=1)

In [8]:
train_test_seurat[columns_remove[0]] = train_test_seurat.iloc[:, 0]

In [9]:
train_test_seurat.shape

(109707, 54)

In [10]:
train_seurat = train_test_seurat.iloc[:90000, :]
test_seurat = train_test_seurat.iloc[90000:, :]

Load train and test data

# 1. Load data and preprocessing

## 1.1 Load train and test data

In [11]:
train = pd.read_csv('./MLR_Project_train.csv')
test = pd.read_csv('./MLR_Project_test.csv')

Show the data format and dimension

In [12]:
train.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,TARGET
0,0,5874.524387,1072.671848,41440.76212,41440.23732,70.405148,7.392780,70.377281,23229.69262,23229.72655,70.378864,7.389173,70.380160,23229.76782,23379.81637,83.418623,11.615135,83.418623,23466.72590,23466.7259,83.418623,7.425618,70.494241,23229.77107,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,70.376262,4898.757333,15165.92759,297487.1654,297487.16540,15165.92759,4898.757333,70.376262,23229.77107,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,64.192982,20.940618,76.070270,23376.73707,23343.13291,77.290965,347.308164,102.380501,24823.08137,24120.94894,332.757607,17.386711,129.622187,23936.99077,21670.19233,71.518948,11.399004,78.006816,26437.161240,23811.09670,141.997532,22.474794,0.013314
1,1,6124.154099,1072.802927,41440.76212,41442.22458,70.456758,7.356050,70.379576,23229.76020,23230.10472,70.273618,7.389813,70.329906,23229.46908,23384.98219,83.418623,11.615135,83.418623,23466.72590,23466.7259,83.418623,7.441223,70.702624,23229.77107,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,70.376262,5100.350569,15788.07683,308790.4312,308790.43120,15788.07683,5100.350569,70.376262,23229.77107,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,2.937218,-14.428210,75.204973,23317.46049,23309.42032,74.880368,689.670872,127.070357,36746.26762,31012.79786,-310.576721,4.532673,-47.045260,22053.23653,14626.73339,48.124991,-99.618253,-115.120518,7705.543821,22665.35143,-377.287072,-73.700375,-0.000448
2,2,5905.732593,1072.802927,41443.14358,41442.30403,70.422472,7.417794,70.376448,23229.48142,23229.61008,70.474265,7.388979,70.397301,23229.83396,23387.39168,83.418623,11.615135,83.418623,23466.72590,23466.7259,83.418623,7.440201,70.686178,23229.77107,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,70.376262,5308.548086,16430.60795,320463.9968,320463.99680,16430.60795,5308.548086,70.376262,23229.77107,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,141.231442,25.855392,75.184756,23248.26780,23307.91084,74.635569,634.523047,71.705965,28917.00549,24632.17456,419.071308,7.403187,118.846496,23430.24573,31251.55292,71.535567,53.482719,106.179152,37586.677270,23251.62576,261.098973,22.565621,0.000244
3,3,6029.325221,1072.802927,41440.76212,41442.25682,70.458533,7.366031,70.379221,23230.08433,23229.87971,70.288944,7.390120,70.370247,23229.81662,23390.02296,83.418623,11.615135,83.418623,23233.34325,23466.7259,83.418623,7.439191,70.692085,23229.77107,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,70.376262,5510.265781,17053.14128,331774.2409,75710.89648,17053.14128,5510.265781,70.376262,23229.77107,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,-1.580478,-7.499746,75.184756,23248.26836,23307.91084,74.635569,583.001082,70.384022,25642.94219,23482.26269,-249.671869,7.388974,49.809681,23193.67720,15867.01579,70.369496,-12.169114,63.930236,10052.351290,23229.64352,-10.549985,4.656636,-0.000628
4,4,6142.360146,1072.802927,41443.14358,41442.46480,70.413623,7.411287,70.376788,23229.70975,23229.82255,70.467206,7.389910,70.386986,23229.87603,23392.66710,70.572881,11.615135,83.418623,23466.72590,23466.7259,83.418623,7.438244,70.680386,23229.77107,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,70.376262,1152.667997,17699.01445,343508.5253,343508.52530,17699.01445,5719.546213,70.376262,23229.77107,23229.77107,70.376262,7.389056,70.376262,23229.77107,23229.77107,149.144800,26.789196,74.904988,23229.77108,23260.32245,70.506980,566.549008,120.694073,25765.82131,24618.79392,363.188022,7.389057,78.826922,23235.86490,31790.06425,120.694068,42.971377,145.572170,37109.895810,24143.94971,188.639704,31.863254,0.003811


In [13]:
test.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,TARGET
0,0,16335.49448,1061.530132,41238.33840,41237.30921,70.432910,7.379175,70.376460,23229.86828,23229.78910,70.397639,7.389850,70.389234,23229.90881,25602.51370,70.835039,7.537712,70.835039,23235.72471,23235.72471,70.703960,7.467846,70.433077,23230.96180,23278.59091,70.835039,7.537712,70.441802,23238.10616,23235.72471,70.703960,7133.700113,22063.32144,422799.6647,232518.5574,11589.95489,3740.045136,1860.710635,4138543.879,422799.6647,22063.32144,137.249904,66395.12033,232518.5574,232518.5574,258.079242,30.226219,75.913405,23229.77107,23229.83301,70.376262,1473.581679,71.758838,23874.69807,23499.73762,-155.498907,7.389056,70.371005,23229.77106,12139.388420,71.758829,14.703740,84.826620,8035.667142,23254.88967,92.945298,12.071364,0.010438
1,1,13166.20458,1061.661211,41238.33840,41238.66844,70.394428,7.378583,70.391576,23229.87463,23230.21191,70.288403,7.392921,70.343310,23229.64193,25607.58558,70.572881,7.452766,70.572881,23240.48762,23240.48762,70.966118,7.469469,70.576552,23230.96180,23235.72471,70.572881,7.452766,70.572881,23280.97236,23240.48762,70.966118,3945.011279,12222.51346,244010.9410,434292.0482,22695.88002,7338.666257,139.167102,1239716.334,244010.9410,12222.51346,792.472120,227216.19080,434292.0482,434292.0482,-147.808334,-20.702534,76.035317,23313.93717,23332.97615,75.852147,1828.336140,140.129854,37438.79508,30329.40335,167.006374,16.564437,34.929416,23264.53239,39398.972520,84.750309,60.805885,155.880284,41154.557460,24005.38063,199.782364,35.714646,-0.000532
2,2,12897.58082,1061.661211,41240.71985,41238.89069,70.476942,7.400935,70.375313,23229.84429,23229.89405,70.469979,7.391477,70.407134,23230.09254,25610.09095,70.572881,7.580185,70.966118,23233.34325,23233.34325,70.572881,7.470161,70.605117,23233.34325,23280.97236,70.966118,7.580185,70.441802,23235.72471,23233.34325,70.572881,995.797053,23323.37355,445692.4097,255411.3025,12850.00699,4148.336212,3120.762743,4161436.624,445692.4097,23323.37355,233.004088,67655.17244,255411.3025,255411.3025,289.867668,32.185249,75.966899,23326.54724,23327.60253,75.649862,1924.171252,111.147939,37691.96604,29511.34868,53.482595,13.727445,165.790143,24499.54560,7180.863302,74.919427,-22.153780,54.137349,6873.937564,23667.70306,74.616186,24.773579,0.000726
3,3,12887.42863,1061.661211,41238.33840,41239.00250,70.412815,7.378490,70.377339,23229.91979,23229.86689,70.283757,7.389189,70.359007,23229.67957,25612.79171,70.572881,7.452766,70.572881,23233.34325,23240.48762,70.966118,7.469667,70.602493,23230.96180,23235.72471,70.572881,7.452766,70.572881,23280.97236,23240.48762,70.966118,4329.804019,13410.04591,265586.1421,88824.3188,23883.41247,7723.458997,1326.699554,1261291.535,265586.1421,13410.04591,1177.264861,228403.72320,455867.2494,455867.2494,-141.955445,-15.581110,75.883198,23230.50078,23310.57289,73.052689,1808.877007,123.263599,31327.33150,25876.72169,86.016065,7.327061,26.252334,22958.21980,39599.779840,122.820383,46.190366,145.600464,39883.925250,24179.10475,145.999434,26.920634,0.001459
4,4,11994.73779,1061.661211,41240.71985,41238.81077,70.481341,7.392913,70.385266,23229.57329,23229.79297,70.450114,7.390389,70.377065,23229.74335,25615.17798,70.572881,7.580185,70.966118,23233.34325,23233.34325,70.572881,7.469671,70.607659,23233.34325,23280.97236,70.966118,7.580185,70.441802,23235.72471,23233.34325,70.572881,1408.908041,24598.30068,468855.4055,278574.2983,14124.93413,4561.447200,4395.689873,4184599.619,468855.4055,24598.30068,175.464314,68930.09957,278574.2983,278574.2983,276.189378,21.701184,76.055710,23354.81165,23345.67063,77.102636,1906.984098,598.329736,40066.97972,36345.65544,-605.228200,-132.958333,-529.575904,12869.21582,6244.418503,-23.787704,-46.168214,-57.984418,-6030.026819,13649.75983,-694.862277,-218.983324,-0.001462


## 1.3 Show the maximum return of train and test

In [14]:
train_max = np.sum(train['TARGET'][train['TARGET']>0])
test_max = np.sum(test['TARGET'][test['TARGET']>0])

print('Maximum return of training set:', train_max)
print('Maximum return of testing set:', test_max)

Maximum return of training set: 195.6927566509
Maximum return of testing set: 55.96225182400002


In [15]:
reg = Ridge(alpha=0.5).fit(pd.DataFrame(train_seurat.iloc[:, :]), train['TARGET'])
pred = reg.predict(pd.DataFrame(train_seurat.iloc[:, :]))

pred_test = reg.predict(pd.DataFrame(test_seurat.iloc[:, :]))

train_res = np.sum(train['TARGET'][pred>0])
test_res = np.sum(test['TARGET'][pred_test>0])

In [16]:
print(f'Train naive random selection percentage return: {train_res/train_max*100}%')
print(f'Test naive random selection percentage return: {test_res/test_max*100}%')

Train naive random selection percentage return: 6.32652526152815%
Test naive random selection percentage return: 4.010962118285209%


### 1.3.1 Remove the Unnamed columns in dataframe

In [17]:
train = train.loc[:, ~train.columns.str.contains('^Unnamed')]
test = test.loc[:, ~test.columns.str.contains('^Unnamed')]

## 1.4 Naive random selection experiment

In [18]:
train_random = 0
for j in range(10000):
    ind = np.random.randint(2, size=train.shape[0])
    train_random = train_random + sum(train['TARGET'][ind>0])

print('Result of train naive random selection:', train_random/10000)

test_random = 0
for j in range(10000):
    ind = np.random.randint(2, size=test.shape[0])
    test_random = test_random + sum(test['TARGET'][ind>0])

print('Result of test naive random selection:', test_random/10000)


Result of train naive random selection: -0.8170112755425528
Result of test naive random selection: -0.6496402211191966


In [19]:
print(f'Train naive random selection percentage return: {(train_random/10000)/train_max*100}%')
print(f'Test naive random selection percentage return: {(test_random/10000)/test_max*100}%')

Train naive random selection percentage return: -0.41749694241368096%
Test naive random selection percentage return: -1.1608543258092974%


## 1.5 Get data shape

In [20]:
# print('Train shape:', train.shape)
# print('Test shape:', test.shape)

## 3.1 Remove extreme Target values

In [21]:
# train = train.sort_values(by=['TARGET'])
# # remove samples with extreme large target values and samples with extreme negative values
# num_remove = 10
# train_remove = train.iloc[num_remove:-num_remove, :]
# # train.shape

## 3.2 Remove extreme features values

In [22]:
# for i in range(train.shape[1]-1):
#     train_remove = train_remove.sort_values(by=[str(i)])
#     num_remove = 5
#     train_remove = train_remove.iloc[num_remove:-num_remove, :]

In [23]:
# train_remove.shape

In [24]:
# figure, ax = plt.subplots(1, 1, figsize = (35, 7))

# train_remove_mean_values = train_remove.mean()
# train_remove_std_values = train_remove.std()

# train_remove_std = (train_remove-train_remove.mean())/train_remove.std()

# # train_remove_std.boxplot()
# # ax.set_xlabel('Features', fontsize=20)
# ax.set_ylabel('Values', fontsize=20)
# ax.set_title('Train standardized features variability', fontsize=25)

# figure.savefig("features_std_dis.png", bbox_inches='tight', dpi=600)

### The devil is in the details, we need to standardize test dataset by using train mean and train standard deviation

In [25]:
# test_std = (test-train_remove_mean_values)/train_remove_std_values

## 5.5 Autoencoder Resnet model

In [27]:
input_features = train_seurat.to_numpy()
output_features = pd.DataFrame(train['TARGET']).to_numpy()

#######
for i in range(input_features.shape[0]):
    input_features[i, :] = (input_features[i, :] - np.mean(input_features[i, :]))/np.std(input_features[i, :])
#######

input_ = np.zeros((input_features.shape[0], 9, 6))
output_ = np.zeros((output_features.shape[0], 9, 6))

for i in range(input_.shape[0]):
    input_[i, :, :] = np.reshape(input_features[i, :], (9, 6))
    
input_features = input_
    
# input_features = torch.tensor(input_features, 1)

X_test = test_seurat.to_numpy()
Y_test = pd.DataFrame(test['TARGET']).to_numpy()

#######
for i in range(X_test.shape[0]):
    X_test[i, :] = (X_test[i, :] - np.mean(X_test[i, :]))/np.std(X_test[i, :])
#######


X_test_ = np.zeros((X_test.shape[0], 9, 6))
for i in range(X_test_.shape[0]):
    X_test_[i, :, :] = np.reshape(X_test[i, :], (9, 6))
X_test = X_test_
    
X_train, X_val, Y_train, Y_val = train_test_split(input_features, output_features, test_size=0.2, random_state=42)

#####
# to calculate returns
train_data, val_data = train_test_split(train, test_size=0.2, random_state=42)
test_data = test
#####

auto_train_max = np.sum(train_data['TARGET'][train_data['TARGET']>0])
auto_val_max = np.sum(val_data['TARGET'][val_data['TARGET']>0])
auto_test_max = np.sum(test['TARGET'][test['TARGET']>0])

print('Train X shape:', X_train.shape)
print('Validation X shape:', X_val.shape)
print('Test X shape:', X_test.shape)

print('Train Y shape:', Y_train.shape)
print('Val Y shape:', Y_val.shape)
print('Test Y shape:', Y_test.shape)

print('train_max:', auto_train_max)
print('val_max:', auto_val_max)
print('test_max:', auto_test_max)

Train X shape: (72000, 9, 6)
Validation X shape: (18000, 9, 6)
Test X shape: (19707, 9, 6)
Train Y shape: (72000, 1)
Val Y shape: (18000, 1)
Test Y shape: (19707, 1)
train_max: 156.45405680890002
val_max: 39.238699842
test_max: 55.96225182400002


In [28]:
train_input = torch.from_numpy(X_train)
train_output = torch.from_numpy(Y_train)
val_input = torch.from_numpy(X_val)
val_output = torch.from_numpy(Y_val)
test_input = torch.from_numpy(X_test)
test_output = torch.from_numpy(Y_test)

train_input = torch.unsqueeze(train_input, 1)
val_input = torch.unsqueeze(val_input, 1)
test_input = torch.unsqueeze(test_input, 1)

train_input = train_input.float()
train_output = train_output.float()
val_input = val_input.float()
val_output = val_output.float()
test_input = test_input.float()
test_output = test_output.float()

input_feature = train_input.shape[1]
output_feature = 1

# print('input_feature:', input_feature)
# print('output_feature:', output_feature)

In [29]:
train_input = train_input.to(device)
train_output = train_output.to(device)
val_input = val_input.to(device)
val_output = val_output.to(device)
test_input = test_input.to(device)
test_output = test_output.to(device)

In [30]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything()

In [35]:
# auto-encoder model
# base model
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 2)
        self.conv2 = nn.Conv2d(16, 32, 2)
        self.maxpool = nn.MaxPool2d(2)
        
        self.linear = nn.Linear(96, 1)
        
        self.linear2 = nn.Linear(input_feature//2, input_feature//4)
        self.linear3 = nn.Linear(input_feature//4, input_feature//16)
        self.linear4 = nn.Linear(input_feature//16, input_feature//16)
        
        self.linear5 = nn.Linear(input_feature//16, input_feature//16)
        self.linear6 = nn.Linear(input_feature//16+input_feature, input_feature//16)
        
        self.batchnorm_1 = nn.BatchNorm1d(input_feature//2)
        self.batchnorm_2 = nn.BatchNorm1d(input_feature//4)
        self.batchnorm_3 = nn.BatchNorm1d(input_feature//16)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.15)
        
        self.softmax = nn.Softmax()
        

    def forward(self, x_):
        x = self.conv1(x_)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = torch.flatten(x, 1)
        x = self.relu(x)
        output = self.linear(x)
        return output.float()

In [36]:
batch_size = 100000
train_ds = TensorDataset(train_input, train_output)
train_dl = DataLoader(train_ds, batch_size= batch_size, shuffle=False)

In [37]:
%matplotlib inline
def fit(num_epochs, model, loss_fn, train_input, train_output, val_input, val_output, test_input, test_output, model_path):
    best_loss = float('inf')
    train_pred_output = []
    val_pred_output = []
    train_error = []
    val_error = []
    test_error = []
    epochs = []
    
    train_returns = []
    val_returns = []
    test_returns = []
    
    train_sum = []
    val_sum = []
    test_sum = []

    for epoch in range(num_epochs):
        for x,y in train_dl:
            model = model.train()
            opt.zero_grad()
            pred = model(x)
            y = torch.reshape(y, (y.shape[0], 1))
            loss = loss_fn(pred, y)
            loss.backward()
            opt.step()

        if epoch % 50 == 0:
            
            model = model.eval()
            
            train_pred = model(train_input)
            train_pred_index = (torch.sign(train_pred)+1)//2
            train_output = torch.reshape(train_output, (train_output.shape[0], 1))
            train_loss = loss_fn(train_output, train_pred)
            # train_loss = loss_fn(train_pred, train_output.long().squeeze())
            train_loss = train_loss.cpu().detach().numpy()
            
            val_pred = model(val_input)
            val_pred_index = (torch.sign(val_pred)+1)//2
            val_output = torch.reshape(val_output, (val_output.shape[0], 1))
            val_loss = loss_fn(val_output, val_pred)
            # val_loss = loss_fn(val_pred, val_output.long().squeeze())
            val_loss = val_loss.cpu().detach().numpy()
        
            test_pred = model(test_input)
            test_pred_index = (torch.sign(test_pred)+1)//2
            test_output = torch.reshape(test_output, (test_output.shape[0], 1))
            test_loss = loss_fn(test_output, test_pred)
            # test_loss = loss_fn(test_pred, test_output.long().squeeze())
            test_loss = test_loss.cpu().detach().numpy()
    
            epochs.append(epoch)
            train_error.append(math.log(train_loss+1))
            val_error.append(math.log(val_loss+1))
            test_error.append(math.log(test_loss+1))
            
#             figure, ax = plt.subplots(1, 2, figsize = (20, 7))
#             ax = ax.flatten()
            
#             figure, ax = plt.subplots(1, 4, figsize = (22, 5))
#             ax = ax.flatten()
            
#             plt.grid(False)
            # train_conf = confusion_matrix(train_output, train_pred_index)
#             g1 = sns.heatmap(train_conf, cmap="YlGnBu",cbar=False, ax=ax[0], annot = True)
#             g1.set_ylabel('True Target')
#             g1.set_xlabel('Predict Target')
#             g1.set_title('Train dataset')

#             plt.grid(False)
            # val_conf = confusion_matrix(val_output, val_pred_index)
#             g2 = sns.heatmap(val_conf, cmap="YlGnBu",cbar=False, ax=ax[1], annot = True)
#             g2.set_ylabel('True Target')
#             g2.set_xlabel('Predict Target')
#             g2.set_title('Val dataset')
            
#             plt.grid(False)
            # test_conf = confusion_matrix(test_output, test_pred_index)
#             g3 = sns.heatmap(test_conf, cmap="YlGnBu",cbar=False, ax=ax[2], annot = True)
#             g3.set_ylabel('True Target')
#             g3.set_xlabel('Predict Target')
#             g3.set_title('Test dataset')
            
            train_pred_np = train_pred_index.cpu().detach().numpy()
            train_output_np = train_output.cpu().detach().numpy()
            val_pred_np = val_pred_index.cpu().detach().numpy()
            val_output_np = val_output.cpu().detach().numpy()
            test_pred_np = test_pred_index.cpu().detach().numpy()
            test_output_np = test_output.cpu().detach().numpy()
            
#             train_max_value = max(max(train_output_np), max(train_pred_np))
#             train_min_value = min(min(train_output_np), min(train_pred_np))
#             val_max_value = max(max(val_output_np), max(val_pred_np))
#             val_min_value = min(min(val_output_np), min(val_pred_np))
#             test_max_value = max(max(test_output_np), max(test_pred_np))
#             test_min_value = min(min(test_output_np), min(test_pred_np))
            
#             ax[0].scatter(train_output_np, train_pred_np, s = 20, alpha=0.3, c='blue')
#             ax[1].scatter(val_output_np, val_pred_np, s = 20, alpha=0.3, c='red')
#             ax[2].scatter(test_output_np, test_pred_np, s = 20, alpha=0.3, c='green')
            
#             ax[0].plot(epochs, train_error, c='blue')
#             ax[0].plot(epochs, val_error, c='red')
#             ax[0].plot(epochs, test_error, c='green')
#             ax[0].set_title('Errors vs Epochs', fontsize=15)
#             ax[0].set_xlabel('Epoch', fontsize=10)
#             ax[0].set_ylabel('Errors', fontsize=10)

#             ax[0].legend(['train', 'valid', 'test'])
            
#             ax[0].set_xlim([train_min_value, train_max_value])
#             ax[0].set_ylim([train_min_value, train_max_value])
#             ax[0].set_title('Trainig data', fontsize=15)
#             ax[0].set_xlabel('Target', fontsize=10)
#             ax[0].set_ylabel('Prediction', fontsize=10)
#             ax[0].plot([train_min_value, train_max_value], [train_min_value, train_max_value], 'k-')
            
#             ax[1].set_xlim([val_min_value, val_max_value])
#             ax[1].set_ylim([val_min_value, val_max_value])
#             ax[1].set_title('Validation data', fontsize=15)
#             ax[1].set_xlabel('Target', fontsize=10)
#             ax[1].set_ylabel('Prediction', fontsize=10)
#             ax[1].plot([val_min_value, val_max_value], [val_min_value, val_max_value], 'k-')
            
#             ax[2].set_xlim([test_min_value, test_max_value])
#             ax[2].set_ylim([test_min_value, test_max_value])
#             ax[2].set_title('Testing data', fontsize=15)
#             ax[2].set_xlabel('Target', fontsize=10)
#             ax[2].set_ylabel('Prediction', fontsize=10)
#             ax[2].plot([test_min_value, test_max_value], [test_min_value, test_max_value], 'k-')
            
#             ax[3].plot(epochs, train_error, c='blue')
#             ax[3].plot(epochs, val_error, c='red')
#             ax[3].plot(epochs, test_error, c='green')
#             ax[3].set_title('Training and Validation error', fontsize=15)
#             ax[3].set_xlabel('Epochs', fontsize=10)
#             ax[3].set_ylabel('MSE error', fontsize=10)
            
#             display.clear_output(wait=True)
#             display.display(pl.gcf())
            
#             print('Epoch ', epoch, 'Train_loss: ', train_loss*1000, ' Validation_loss: ', val_loss*100, ' Test_loss: ', test_loss*100)
            # print(train_pred_np.shape, train_pred_np)
            # print(train_pred, train_pred_np)
            train_pred_np = np.squeeze(train_pred_np)
            val_pred_np = np.squeeze(val_pred_np)
            test_pred_np = np.squeeze(test_pred_np)
            
            train_res = np.sum(train_data['TARGET'][train_pred_np>0])
            train_output_check = np.squeeze(train_output_np)
            train_check = np.sum(train_data['TARGET'][train_output_check>0])
            
            val_res = np.sum(val_data['TARGET'][val_pred_np>0])
            val_output_check = np.squeeze(val_output_np)
            val_check = np.sum(val_data['TARGET'][val_output_check>0])
            
            test_res = np.sum(test_data['TARGET'][test_pred_np>0])
            test_output_check = np.squeeze(test_output_np)
            test_check = np.sum(test_data['TARGET'][test_output_check>0])
            
#             train_returns.append(train_res)
#             val_returns.append(val_res)
#             test_returns.append(test_res)
            
#             ax[1].plot(epochs, train_returns, c='blu`e')
#             ax[1].plot(epochs, val_returns, c='red')
#             ax[1].plot(epochs, test_returns, c='green')
#             ax[1].legend(['train', 'valid', 'test'])
#             ax[1].set_title('Return vs Epochs', fontsize=15)
#             ax[1].set_xlabel('Epoch', fontsize=10)
#             ax[1].set_ylabel('Returns', fontsize=10)

#             display.clear_output(wait=True)
#             display.display(pl.gcf())
            
            train_sum.append(train_res)
            val_sum.append(val_res)
            test_sum.append(test_res)
            # print(f'Checks: {train_check/auto_train_max*100}%, {val_check/auto_val_max*100}%, {test_check/auto_test_max*100}%')
#             print(f'Maximum sum train return {train_res}, Total train return: {auto_train_max}, Maximum train percentage return: {train_res/auto_train_max*100}%')
#             print(f'Maximum sum train return {val_res}, Total train return: {auto_val_max}, Maximum train percentage return: {val_res/auto_val_max*100}%')
#             print(f'Maximum sum test return {test_res}, Total test return: {auto_test_max}, Maximum test percentage return: {test_res/auto_test_max*100}%')
#             print('Epoch:', epoch, 'Train loss:', train_loss, 'Val loss:', val_loss, 'Test loss:', test_loss)
            print(f'Epoch: {epoch}, Train loss: {train_loss}, Train return: {train_res/auto_train_max*100}%, Val loss: {val_loss}, Val return: {val_res/auto_val_max*100}%, Test loss: {test_loss}, Test return: {test_res/auto_test_max*100}%')
        
            if val_loss < best_loss:
                torch.save(model.state_dict(), model_path)
                best_loss = val_loss
                
#             train_pred_output.append([train_pred.cpu().detach().numpy(), train_output.cpu().detach().numpy()])
#             val_pred_output.append([val_pred.cpu().detach().numpy(), val_output.cpu().detach().numpy()])
    return train_sum, val_sum, test_sum


In [38]:
num_epochs = 20000
learning_rate = 0.001
loss_fn = F.mse_loss

seed_everything()

model = Autoencoder()
model = model.to(device)
opt = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
train_sum_1, val_sum_1, test_sum_1, train_conf_1, val_conf_1, test_conf_1 = fit(num_epochs, model, loss_fn, train_input, train_output, val_input, val_output, test_input, test_output, 'model_path_cnn')
# fig.savefig("auto_encoder.png", bbox_inches='tight', dpi=600)

Epoch: 0, Train loss: 0.00842222385108471, Train return: -0.6734364015162209%, Val loss: 0.008379928767681122, Val return: -0.7255568205531266%, Test loss: 0.008193804882466793, Test return: -0.9211383123416649%
Epoch: 50, Train loss: 0.004782435018569231, Train return: -1.6590978959213156%, Val loss: 0.004779001232236624, Val return: -2.0249661181421565%, Test loss: 0.004752246662974358, Test return: 0.6427081492917412%
Epoch: 100, Train loss: 0.0033749272115528584, Train return: -1.4823133729492781%, Val loss: 0.0033766848500818014, Val return: -2.027651507832037%, Test loss: 0.0033532101660966873, Test return: 0.6526274141874032%
Epoch: 150, Train loss: 0.002630552975460887, Train return: -1.4579789081379864%, Val loss: 0.002631840528920293, Val return: -1.7021610774297176%, Test loss: 0.002607585396617651, Test return: 0.6699481771732899%
Epoch: 200, Train loss: 0.002166937105357647, Train return: -1.3340090258888442%, Val loss: 0.002166402991861105, Val return: -1.587414426339595%

KeyboardInterrupt: 

In [ ]:
# model = Autoencoder_model()
# model.load_state_dict(torch.load(model_path))
# model.eval()